In [ ]:
!pip install mlflow-skinny

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [12]:
%cd /content/gdrive/MyDrive/cyber2a
!git clone https://github.com/cyber2a/Cyber2A-RTS-ToyModel.git

/content/gdrive/MyDrive/cyber2a


In [20]:
%cd /content/gdrive/MyDrive/cyber2a/Cyber2A-RTS-ToyModel

/content/gdrive/MyDrive/cyber2a/Cyber2A-RTS-ToyModel


In [14]:
import sys
sys.path.append("//content/gdrive/MyDrive/cyber2a/Cyber2A-RTS-ToyModel/toy_model")



In [31]:
import torch
from dataset import RTSDataset
from transforms import get_transform
from utils import collate_fn
from model import get_model_instance_segmentation
from engine import evaluate, train_one_epoch

In [24]:
    # Set device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [30]:
dataset = RTSDataset("data/coco_rts_train.json", get_transform(train=True))
dataset_test = RTSDataset("data/coco_rts_valtest.json", get_transform(train=False))

# Create data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, collate_fn=collate_fn
)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn
)

# Initialize model
model = get_model_instance_segmentation(num_classes=2)
model.to(device)

# Set up optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)




In [32]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)


//content/gdrive/MyDrive/cyber2a/Cyber2A-RTS-ToyModel/toy_model/engine.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [  0/378]  eta: 0:20:06  lr: 0.000018  loss: 4.9414 (4.9414)  loss_classifier: 0.9866 (0.9866)  loss_box_reg: 0.1537 (0.1537)  loss_mask: 3.0828 (3.0828)  loss_objectness: 0.7001 (0.7001)  loss_rpn_box_reg: 0.0183 (0.0183)  time: 3.1911  data: 0.0696  max mem: 2430
Epoch: [0]  [ 10/378]  eta: 0:05:10  lr: 0.000151  loss: 3.0224 (3.4364)  loss_classifier: 0.8652 (0.7522)  loss_box_reg: 0.1537 (0.1615)  loss_mask: 1.8044 (2.1947)  loss_objectness: 0.1597 (0.3090)  loss_rpn_box_reg: 0.0124 (0.0191)  time: 0.8435  data: 0.0199  max mem: 4503
Epoch: [0]  [ 20/378]  eta: 0:04:19  lr: 0.000283  loss: 1.7815 (2.5181)  loss_classifier: 0.3309 (0.4947)  loss_box_reg: 0.1492 (0.1597)  loss_mask: 1.0324 (1.5549)  loss_objectness: 0.1540 (0.2841)  loss_rpn_box_reg: 0.0151 (0.0248)  time: 0.6007  data: 0.0146  max mem: 4503
Epoch: [0]  [ 30/378]  eta: 0:03:58  lr: 0.000416  loss: 1.1528 (2.0450)  loss_classifier: 0.1703 (0.3865)  loss_box_reg: 0.1474 (0.1511)  loss_mask: 0.7118 (1.2646) 